###### References: 
- https://docs.python.org/3/tutorial/datastructures.html   
- Fluent Python by Luciano Ramalho. Chapter 3: Dictionaries and Sets

# Generic Mapping Types
Hash tables are the engines behind Python's high-performance `dicts`.


In [1]:
from collections import abc
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

In [2]:
isinstance(my_dict, abc.MutableMapping)

True

https://docs.python.org/3/glossary.html#term-hashable

An object is _hashable_ if it has a hash value which never changes during its lifetime (it needs a `__hash__()` method), and can be compared to other objects (it needs an `__eq__()` method). Hashable objects which compare equal must have the same hash value.

In [3]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [4]:
tl = (1, 2, [30, 40])
try:
    hash(tl)
except TypeError as e:
    print(e)

unhashable type: 'list'


In [5]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

Building `dict`:

In [6]:
a = dict(one=1, two=2, three=3)
b = {'three': 3, 'two': 2, 'one': 1}
c = dict([('two', 2), ('one', 1), ('three', 3)])
d = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## dict Comprehensions

In [7]:
dial_codes = [                                                  # <1>
    (880, 'Bangladesh'),
    (55,  'Brazil'),
    (86,  'China'),
    (91,  'India'),
    (62,  'Indonesia'),
    (81,  'Japan'),
    (234, 'Nigeria'),
    (92,  'Pakistan'),
    (7,   'Russia'),
    (1,   'United States'),
]


country_dial = {country: code for code, country in dial_codes}
country_dial

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [8]:
{code: country.upper() 
    for country, code in sorted(country_dial.items())
    if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

## Common Mapping Methods
### Handing Missing Keys with `setdefault`
Build an index mapping `word` -> `list` of occurrences:

In [9]:
dic= {'IND': 'India', 'USA': 'United States of America', 'AUS': 'Australia' }

#key is not present
dic.get('SIN','Not found')

'Not found'

Bad example when updating the value found:

In [10]:
dic['SIN'] = 'Singapore'
dic

{'IND': 'India',
 'USA': 'United States of America',
 'AUS': 'Australia',
 'SIN': 'Singapore'}

In [11]:
add_list = [('IND', 'India'), ('PAK', 'Pakistan')]
for key, val in  add_list:
    dic.setdefault(key, val)
dic

{'IND': 'India',
 'USA': 'United States of America',
 'AUS': 'Australia',
 'SIN': 'Singapore',
 'PAK': 'Pakistan'}

## `defaultdict` - Flexible Key  Lookup

In [12]:
import collections

s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]

d = collections.defaultdict(list)

for k, v in s:

    d[k].append(v)

d

defaultdict(list, {'yellow': [1, 3], 'blue': [2, 4], 'red': [1]})

### the `__missing__` method
If a subclass of `dict` provide a `__missing__` method, the standard `dict.__getitem__` will call  it whenever a key is not found, instead of raising `KeyError`.

Mapping where keys are converted to `str` when looked up:

In [13]:
class StrKeyDict0(dict):  # inherits from dict
    """ Keys are converted to str when looked up.
    """
    def __missing__(self, key):
        if isinstance(key, str):  # check whether key is already a string
            raise KeyError(key)  # raise error if missing
        return self[str(key)]  # build str from key and look it up

    def get(self, key, default=None):
        try:
            return self[key]  # delegates to __getitem__
        except KeyError:
            return default  # if __missing__ failed return to default

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()  # search for unmodified key and str built

Tests for item retrieval using `d[key]` notation:

In [14]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [15]:
d[4]

'four'

In [16]:
d[1]

KeyError: '1'

Tests for item retrieval using `d.get(key)` notation:

In [17]:
d.get('2')

'two'

In [18]:
d.get(4)

'four'

In [19]:
d.get(1, 'N/A')

'N/A'

Tests for the `in` operator:

In [20]:
2 in d

True

In [21]:
1 in d

False

## Variations of `dict`
    * collections.OrderedDict
    * collections.ChainMap

In [22]:
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))

    * collections.Counter

In [23]:
ct = collections.Counter('gallahad') 
ct

Counter({'g': 1, 'a': 3, 'l': 2, 'h': 1, 'd': 1})

In [24]:
ct.update('python')
ct

Counter({'g': 1,
         'a': 3,
         'l': 2,
         'h': 2,
         'd': 1,
         'p': 1,
         'y': 1,
         't': 1,
         'o': 1,
         'n': 1})

In [25]:
ct.most_common(2)

[('a', 3), ('l', 2)]

In [26]:
ct['i']

0

    * collections.UserDict
## Subclassing `UserDict`
 * stores all keys as `str`
 * subclasses `MutableMapping`, able to load instances from other mappings
 * inherits `Mapping.get`
Make sure any keys added to the mapping are stored as `str`:

In [27]:
class StrKeyDict(collections.UserDict):  # extends UserDict

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data  # can assume all sotred keys are str

    def __setitem__(self, key, item):
        self.data[str(key)] = item   # converts any key to str

Test for initializer: keys are converted to `str`.

In [28]:
d = StrKeyDict([(2, 'two'), ('4', 'four')])
sorted(d.keys())

['2', '4']

Tests for item retrieval using `d[key]` notation:

In [29]:
 d['2']

'two'

In [30]:
d[4]

'four'

In [31]:
d[1]

KeyError: '1'

Tests for item retrieval using `d.get(key)` notation:

In [32]:
d.get('2')

'two'

In [33]:
d.get(4)

'four'

In [34]:
d.get(1, 'N/A')

'N/A'

Tests for the `in` operator:

In [35]:
2 in d

True

In [36]:
1 in d

False

Tests for update using a `dict` or a sequence of pairs:

In [37]:
d.update({6:'six', '8':'eight'})
sorted(d.keys())

['2', '4', '6', '8']

In [38]:
d.update([(10, 'ten'), ('12', 'twelve')])
sorted(d.keys())

['10', '12', '2', '4', '6', '8']

In [39]:
d.update([1, 3, 5])

TypeError: cannot unpack non-iterable int object

## Immutable Mappings
`MappingProxyType` returns a `mappyproxy` instance that is a read-only but dynamic view of the original mapping.  
Updates to original mapping can be seen, but changes can not be made through it.

In [40]:
from types import MappingProxyType

d = {1: 'A'}
d_proxy  = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [41]:
d_proxy[1]

'A'

In [42]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [43]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [44]:
d_proxy[2]

'B'

# Set Theory
Collection of unique objects. 

Immutable sibling `frozenset`.

Basic use case is removing duplication

In [45]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

In [46]:
list(set(l))

['spam', 'eggs']

Set elements must be hashable. The `set` type is not hashable, but `frozenset` is.

Count occurrences of *needles* in a *haystack*:

    found = len(needles & haystack)

any iterable types:

    found = len(set(needles) & set(haystack))

another way:

    found = len(set(needles).intersection(haystack))


## `set` literals
Syntax of `set` literals looks exactly like the math notation with exception of the empty `set`, `{}` creates an empty dict.

In [47]:
s = {1}
type(s)

set

In [48]:
s

{1}

In [49]:
s.pop()

1

In [50]:
s

set()

Literal `set` syntax is both faster and more readable than calling the constructor.

In [51]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## Set Comprehensions

In [52]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

## Set Operations
Infix operations require both operands to be the same. But other methods take one or more iterable arguments. e.g.:

    a.union(b, c, d)

where `a` must be a `set` but `b`, `c` and `d` can be iterables of any type.

# `dict` and `set` under the hood
 * How efficient are Python `dict` and `set`?
 * Why are they unordered?
 * Why can't we use any Python object as a `dict` key or `set` element?
 * Why does the order of the `dict` keys or `set` elements depend on insertion order, and may change during the lifetime of the structure?
 * Why is it bad to add items to a `dict` or `set` while iterating through it?
 
## Hash Tables in Dictionaries
    found = 0
    for n in needles:
        if n in haystack:
         found += 1
##### Table 3-6. Total time using in operator to search for 1,000 keys in haystack of 5 sizes.
| len of haystack | dict time | set time | set& time | list time | 
| :---   | :---  | :---  | :---  | :---  |
| 1,000 | 0.000202s | 0.000143s | 0.000087s | 0.010556s |
| 10,000 | 0.000140s | 0.000147s | 0.000092s | 0.086586s |
| 100,000 | 0.000228s | 0.000241s | 0.000163s | 0.871560s |
| 1,000,000 | 0.000290s | 0.000332s | 0.000250s | 9.189616s |
| 10,000,000 | 0.000337s | 0.000387s | 0.000314s | 97.94806s |


## Hash Tables in Dictionaries
https://en.wikipedia.org/wiki/Hash_table

A hash table is a sparse array.  
The cells in a hash table are often called 'buckets', of the same size, containing two fields: a reference to the key and a reference to the value.   

Python tries to keep at least 1/3 of the buckets empty.  

The hash value of the item key is first calculated before inserting the item into a hash table.   

### Hashes and equality
If two objects compare equal, their has values must also be equal.

E.g. Because `1 == 1.0` is true, `hash(1) == hash(1.0)` must also be true.

Also, to be effective, hash values should scatter around the index space as much as possible. I.e. objects that are similar but not equal should have hash values that differ widely.

### The hash  table algorithm
To fetch value at  `my_dict[search_key]`, Python calls `hash(search_key)` to obtain the *hash value*, and use the  least significant  bits of that number as an offset to look up a bucket in the hash table. `KeyError` is raised if bucket is empty, otherwise `found_key:found_value` pair is checked if `search_key == found_key`, and `found_value` is returned.

If `search_key` and `found_key` do not match, there is a *hash_collision*. In which a different hash is calculated using different bits to assign to different bucket.

### Practical consequences of `dict`
#### Keys must be hashable objects
 * It supports the `hash()` function via the `__hash()__` method that always return the same value over the lifetime of the object.
 * It supports equality via an `__eq__()` method.
 * If `a == b`, then `hash(a) == hash(b)`
#### `dicts` have significant memory overhead
For large quantities of records, better to store them in a list of tuples or named tuples instead of using dictionaries in JSON style, with one `dict` per record.

This reduces overhead of one hash table per record and storing of field names per record.

This is balancing Optimization v.s. Maintainability.

#### Key search  is very fast
#### Key  ordering depends on  insertion order
Adding items to a dict may change the order of existing keys.

### Practical consequences of `set`
The `set` and `frozenset` are also implemented with hash table, except that each bucket holds only references to the element.
 * Set elements mush be hashable objets
 * Sets have a significant memory overhead
 * Membership testing is very efficient
 * Element ordering depends on insertion order
 *  Adding elements to a set may change the order of other elements

In [53]:
dial_codes = [
    (880, 'Bangladesh'),
    (55,  'Brazil'),
    (86,  'China'),
    (91,  'India'),
    (62,  'Indonesia'),
    (81,  'Japan'),
    (234, 'Nigeria'),
    (92,  'Pakistan'),
    (7,   'Russia'),
    (1,   'United States'),
]

In [54]:
country_dial = {country: code for code, country in dial_codes}
country_dial

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [55]:
{code: country.upper() 
    for country, code in sorted(country_dial.items())
    if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

In [56]:
from random import shuffle
shuffle(dial_codes)
country_dial = {country: code for code, country in dial_codes}
country_dial

{'Japan': 81,
 'Nigeria': 234,
 'United States': 1,
 'Pakistan': 92,
 'Russia': 7,
 'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'Indonesia': 62,
 'India': 91}